# Another Instagram Filter (AI-Filter)

The main and only script of the project

## Camera Check

Run this only from JETSON NANO

In [1]:
!ls -ltrh /dev/video*

crw-rw----+ 1 root video 81, 0 mars  30 19:30 /dev/video0


## Import Dependencies

In [2]:
import cv2
import mediapipe as mp
import math
from math import hypot
import numpy as np
from statistics import mean
import time

##  Declare Filter Functions

In [3]:
# Negative Filter
def negativeFilter(img):
    neg = cv2.bitwise_not(img)
    return neg

# A colored sketch filter
def colorSketchFilter(img, shade_factor = 0.07):
    gray, color = cv2.pencilSketch(img, sigma_s=10, sigma_r=0.07, shade_factor = shade_factor) 
    return  color

# Change brightness when no filter is applied
def changeBright(img, intensity ):
    bright = cv2.convertScaleAbs(img, beta=intensity)
    return bright

## Main Script

In [ ]:
# Initialize landmarks for hand
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

filter_val=350
filter_intensity=0

angle_max = 250
k = 1.8
angle = 120
val = 100
r_dial = 50

mode = 0

# filter parameters
brightness = 0
shade_factor = 0.07

#fourcc = cv2.VideoWriter_fourcc(*'XVID') ## uncomment to save video
#out = cv2.VideoWriter('Output/demo.avi', fourcc, 20.0, (640,  480)) ## uncomment to save video
cap = cv2.VideoCapture(0)

while True:
    success,img = cap.read()
    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    # Different modes, 0 = No Filter, 1 = Sketch filter, 2 = Negative Filter
    if mode == 0:
        img = changeBright(img, brightness)
    elif mode == 1:    
        img = colorSketchFilter(img, shade_factor)
    elif mode == 2:
        img = negativeFilter(img)
        img = changeBright(img, brightness)
    
    # Add all the landmarks of a hand to a list and draw them
    ls_lm = []
    if results.multi_hand_landmarks:
        for landmark_all in results.multi_hand_landmarks:
            for id,lm in enumerate(landmark_all.landmark):
                h,w,_ = img.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                ls_lm.append([id,cx,cy])
            mpDraw.draw_landmarks(img,landmark_all,mpHands.HAND_CONNECTIONS)
    
    # If hand is detected
    if ls_lm != []:
        x_0,y_0 = ls_lm[0][1],ls_lm[0][2] # wrist
        x_4,y_4 = ls_lm[4][1],ls_lm[4][2] # thumb tip
        x_5,y_5 = ls_lm[5][1],ls_lm[5][2] # index finger mcp
        x_8,y_8 = ls_lm[8][1],ls_lm[8][2] # index finger tip
        x_9,y_9 = ls_lm[9][1],ls_lm[9][2] # middle finger mcp
        x_12,y_12 = ls_lm[12][1],ls_lm[12][2] # middle finger tip
        x_16,y_16 = ls_lm[16][1],ls_lm[16][2] # ring finger tip
        x_20,y_20 = ls_lm[20][1],ls_lm[20][2] # pinky finger tip
        
        # set palm senter to middle finger mcp. Makes sense in this case 
        x_palm = x_9
        y_palm = y_9
        
        # Calculate the mean distance between each finger and the palm
        s_4 = hypot(x_4-x_palm, y_4-y_palm)
        s_8 = hypot(x_8-x_palm, y_8-y_palm)
        s_12 = hypot(x_12-x_palm, y_12-y_palm)
        s_16 = hypot(x_16-x_palm, y_16-y_palm)
        s_20 = hypot(x_20-x_palm, y_20-y_palm)
        s_mean = mean([s_4, s_8, s_12, s_16, s_20])
        
        # Creates a dial to adjust the filter intensity when fingers are 
        # sufficiently close to each other
        if (s_mean <= r_dial + 10) and (s_mean >= r_dial -10):
            x_bar = x_4-x_12
            y_bar = y_4-y_12
            z_bar = hypot(x_bar,y_bar) + 1
            angle_bar = math.degrees(np.arcsin(x_bar/z_bar))
            val = int((angle_bar*2.8/angle_max)*250)
            cv2.circle(img, (x_palm, y_palm ),r_dial,(153,255,153), 5)
            if mode == 0 or mode == 2:
                brightness = np.interp(val,[0,250],[-100,100])
            if mode == 1:
                shade_factor = np.interp(val,[0,250], [0.01, 0.1])
        
        # Change filter when you close your finger tips
        if s_mean < 25:
            time.sleep(0.3) # add delay or filters will just swap too quickly
            mode += 1
            if mode > 2:
                mode = 0
        
        filter_val=np.interp(val,[0,250],[350,100])
        filter_intensity=np.interp(val,[0,250],[0,100])

    cv2.rectangle(img,(550,100),(600,350),(153,255,153),4)
    cv2.rectangle(img,(550,int(filter_val)),(600,350),(153,255,153),cv2.FILLED)
    cv2.putText(img,f"Filter Intensity: {int(filter_intensity)}%",(300,50), cv2.FONT_HERSHEY_DUPLEX,1,(153,255,153),2)
 #   out.write(img) #Uncomment to film
    cv2.imshow('AI Filter',img)
    
    # Press "q" to quit
    if cv2.waitKey(1) & 0xff==ord('q'):
        break
        
# close and destruct camera      
out.release()
cap.release()     
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### Stop Camera and Close All Windows

Run this cell in case the video capture bugs and you can't close the image window

In [ ]:
out.release()
cap.release()  
cv2.destroyAllWindows()